## Chapter 4: Building Interactive Web Pages with Razor

In the previous chapter, you built your first MVC application and learned the basics of controllers, actions, and views. But modern web applications require more than simple pages—they need reusable components, consistent layouts, and dynamic content. Razor, the view engine in ASP.NET Core, provides powerful features to build interactive and maintainable web pages. In this chapter, we'll dive deep into layout pages, partial views, view components, Tag Helpers, and the various ways to pass data from controllers to views. By the end, you'll be able to create sophisticated, modular views that are easy to manage and extend.

### 4.1 Layout Pages and `_ViewStart.cshtml`

Most websites share common elements across pages: a header with navigation, a footer, and maybe a sidebar. Repeating this markup in every view would be tedious and error-prone. Layout pages solve this by providing a template that other views can wrap themselves in.

#### Creating a Layout Page

A layout page is a Razor view (`.cshtml`) that defines the overall structure of your pages. It contains placeholders where the content of individual views will be inserted. By convention, layout pages are placed in the `Views/Shared` folder and often named `_Layout.cshtml`.

Here's a typical `_Layout.cshtml`:

```html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="utf-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <title>@ViewBag.Title - MyApp</title>
    <link rel="stylesheet" href="~/lib/bootstrap/dist/css/bootstrap.min.css" />
    <link rel="stylesheet" href="~/css/site.css" asp-append-version="true" />
</head>
<body>
    <header>
        <nav class="navbar navbar-expand-sm navbar-toggleable-sm navbar-light bg-white border-bottom box-shadow mb-3">
            <div class="container-fluid">
                <a class="navbar-brand" asp-area="" asp-controller="Home" asp-action="Index">MyApp</a>
                <button class="navbar-toggler" type="button" data-bs-toggle="collapse" data-bs-target=".navbar-collapse">
                    <span class="navbar-toggler-icon"></span>
                </button>
                <div class="navbar-collapse collapse d-sm-inline-flex justify-content-between">
                    <ul class="navbar-nav flex-grow-1">
                        <li class="nav-item">
                            <a class="nav-link text-dark" asp-area="" asp-controller="Home" asp-action="Index">Home</a>
                        </li>
                        <li class="nav-item">
                            <a class="nav-link text-dark" asp-area="" asp-controller="Products" asp-action="Index">Products</a>
                        </li>
                    </ul>
                </div>
            </div>
        </nav>
    </header>
    <div class="container">
        <main role="main" class="pb-3">
            @RenderBody()
        </main>
    </div>

    <footer class="border-top footer text-muted">
        <div class="container">
            &copy; 2026 - MyApp - <a asp-area="" asp-controller="Home" asp-action="Privacy">Privacy</a>
        </div>
    </footer>

    <script src="~/lib/jquery/dist/jquery.min.js"></script>
    <script src="~/lib/bootstrap/dist/js/bootstrap.bundle.min.js"></script>
    <script src="~/js/site.js" asp-append-version="true"></script>
    @await RenderSectionAsync("Scripts", required: false)
</body>
</html>
```

**Key elements:**
- `@RenderBody()` – This is where the content from the specific view will be injected. Every layout must call `RenderBody()` exactly once.
- `@RenderSectionAsync("Scripts", required: false)` – Defines a named section that views can optionally provide content for. Here, it allows views to add scripts at the end of the page. The `required: false` means it's optional; if a view doesn't define the section, nothing is rendered. You can also have required sections that must be defined.
- Tag Helpers like `asp-controller`, `asp-action` generate correct URLs based on routing.

#### Using `_ViewStart.cshtml`

The `_ViewStart.cshtml` file is a special Razor view that runs before any view. It's typically used to set the default layout for all views in a folder. By placing it in the `Views` folder, it applies to all views in that folder and its subfolders.

**Example `_ViewStart.cshtml`:**
```csharp
@{
    Layout = "_Layout";
}
```

You can override the layout for a specific view by setting `Layout` inside that view:

```csharp
@{
    Layout = "_AlternateLayout";
}
```

You can also have multiple `_ViewStart.cshtml` files in subfolders to set different layouts for different areas of your application.

#### Sections in Detail

Sections allow views to inject content into specific parts of the layout. For example, a view might want to add page-specific CSS or scripts.

**Defining a section in a view:**
```html
@section Scripts {
    <script>
        $(document).ready(function() {
            alert('Page loaded!');
        });
    </script>
}
```

If the layout marks a section as required (`required: true`) and a view doesn't provide it, an exception is thrown at runtime.

**Why layouts matter:** They enforce consistency, reduce duplication, and make site-wide changes (like updating a footer) trivial—just edit one file.

---

### 4.2 Partial Views

Partial views are smaller, reusable pieces of Razor markup that can be embedded within other views. They are ideal for rendering components like a product card, a comment list, or a login form that appear in multiple places.

#### Creating a Partial View

Partial views are just like regular views but are usually named with a leading underscore to indicate they are not meant to be directly requested. Place them in the `Shared` folder or in a controller-specific folder.

**Example: `_ProductCard.cshtml`**
```html
@model Product

<div class="card" style="width: 18rem;">
    <div class="card-body">
        <h5 class="card-title">@Model.Name</h5>
        <h6 class="card-subtitle mb-2 text-muted">@Model.Price.ToString("C")</h6>
        <p class="card-text">@Model.Description</p>
        <a asp-action="Details" asp-route-id="@Model.Id" class="btn btn-primary">View Details</a>
    </div>
</div>
```

#### Rendering a Partial View

You can render a partial view in several ways:

**Using the `<partial>` Tag Helper (recommended):**
```html
<partial name="_ProductCard" model="product" />
```

**Using `Html.PartialAsync` (for synchronous rendering in async context):**
```csharp
@await Html.PartialAsync("_ProductCard", product)
```

**Using `Html.RenderPartialAsync` (writes directly to the response stream, more efficient but less flexible):**
```csharp
@{
    await Html.RenderPartialAsync("_ProductCard", product);
}
```

The `<partial>` tag helper is concise and feels like natural HTML.

#### Passing Data to Partial Views

You can pass a model explicitly, as shown above. If you don't pass a model, the partial inherits the parent view's `Model`. You can also pass additional data via `ViewData`:

```html
<partial name="_ProductCard" model="product" view-data="someViewData" />
```

**Use cases:** Reusable UI components, breaking down complex views, and improving maintainability.

---

### 4.3 View Components

Partial views are great for static, simple reuse, but sometimes you need a component that has its own logic—for example, a shopping cart summary that calculates the total, or a dynamic menu that fetches data from a database. **View Components** are the perfect solution. They are like mini-controllers: they have their own logic, can use dependency injection, and return a view.

#### Creating a View Component

A view component is a class that:
- Derives from `ViewComponent` (or is decorated with `[ViewComponent]` attribute).
- Defines an `Invoke` or `InvokeAsync` method that returns `IViewComponentResult`.
- Can use dependency injection via its constructor.
- Its views are placed in `Views/Shared/Components/<ComponentName>/` or `Views/<Controller>/Components/<ComponentName>/`.

**Example: Cart Summary View Component**

First, create a class `CartSummaryViewComponent` in a `ViewComponents` folder (or anywhere in your project):

```csharp
using Microsoft.AspNetCore.Mvc;
using System.Threading.Tasks;

namespace MyFirstMvcApp.ViewComponents
{
    public class CartSummaryViewComponent : ViewComponent
    {
        // This would normally come from a service
        public async Task<IViewComponentResult> InvokeAsync()
        {
            // Simulate fetching cart data
            var itemCount = await GetCartItemCountAsync();
            var total = await GetCartTotalAsync();

            var model = new CartSummaryModel
            {
                ItemCount = itemCount,
                Total = total
            };

            return View(model); // default view: Default.cshtml
        }

        private Task<int> GetCartItemCountAsync() => Task.FromResult(3);
        private Task<decimal> GetCartTotalAsync() => Task.FromResult(129.97m);
    }

    public class CartSummaryModel
    {
        public int ItemCount { get; set; }
        public decimal Total { get; set; }
    }
}
```

Now create the view for this component. Create a folder `Views/Shared/Components/CartSummary` and add a file `Default.cshtml`:

```html
@model CartSummaryModel

<li class="nav-item">
    <a class="nav-link text-dark" asp-controller="Cart" asp-action="Index">
        Cart (@Model.ItemCount items, @Model.Total.ToString("C"))
    </a>
</li>
```

#### Invoking a View Component

You can invoke a view component from any view using:

```html
@await Component.InvokeAsync("CartSummary")
```

Or using a Tag Helper (if registered in `_ViewImports.cshtml`):

```html
<vc:cart-summary />
```

The Tag Helper version uses kebab-case for component and parameter names.

#### Passing Parameters

View components can accept parameters. Modify the `InvokeAsync` method:

```csharp
public async Task<IViewComponentResult> InvokeAsync(bool showTotal)
{
    // use showTotal to conditionally display total
}
```

Then pass parameters:

```html
@await Component.InvokeAsync("CartSummary", new { showTotal = true })
```

Or with Tag Helper:

```html
<vc:cart-summary show-total="true" />
```

**When to use View Components:**
- Reusable, self-contained widgets with logic.
- Dynamic navigation menus.
- Sidebars with recent posts or comments.
- Anywhere you'd use a partial but need server-side logic.

---

### 4.4 Passing Data from Controller to View

Controllers need to pass data to views. ASP.NET Core provides several mechanisms, each suited for different scenarios.

#### 1. Strongly-Typed Models (Recommended)

This is the cleanest and most maintainable approach. You define a model class, populate it in the controller, and pass it to the view.

**Controller:**
```csharp
public IActionResult Index()
{
    var products = _productService.GetAll();
    return View(products); // passes List<Product> as model
}
```

**View:**
```html
@model IEnumerable<Product>

@foreach (var product in Model)
{
    <div>@product.Name</div>
}
```

**Advantages:**
- IntelliSense support.
- Compile-time checking.
- Clear contract between controller and view.

#### 2. ViewData

`ViewData` is a dictionary (key/value) that lives only for the current request. It's useful for passing small amounts of data that don't warrant a full model, like page titles or dropdown options.

**Controller:**
```csharp
public IActionResult Index()
{
    ViewData["Title"] = "Product List";
    ViewData["ShowDiscount"] = true;
    return View();
}
```

**View:**
```html
<h1>@ViewData["Title"]</h1>
@if (ViewData["ShowDiscount"] as bool? == true)
{
    <p>Special discounts available!</p>
}
```

**Drawbacks:**
- No compile-time checking; keys are strings.
- Requires casting.

#### 3. ViewBag

`ViewBag` is a dynamic wrapper around `ViewData`. It provides a more convenient syntax but still lacks compile-time safety.

**Controller:**
```csharp
public IActionResult Index()
{
    ViewBag.Title = "Product List";
    ViewBag.ShowDiscount = true;
    return View();
}
```

**View:**
```html
<h1>@ViewBag.Title</h1>
@if (ViewBag.ShowDiscount)
{
    <p>Special discounts available!</p>
}
```

`ViewBag` is easier to write but can lead to runtime errors if property names are misspelled. It's best used for simple, one-off data.

#### 4. TempData

`TempData` is similar to `ViewData` but persists data across **two requests**. It's implemented using cookies or session state and is ideal for passing messages after a redirect (e.g., "Product added successfully").

**Controller:**
```csharp
[HttpPost]
public IActionResult Create(Product product)
{
    if (ModelState.IsValid)
    {
        _productService.Add(product);
        TempData["SuccessMessage"] = "Product created successfully!";
        return RedirectToAction(nameof(Index));
    }
    return View(product);
}
```

**In the Index view (after redirect):**
```html
@if (TempData["SuccessMessage"] != null)
{
    <div class="alert alert-success">@TempData["SuccessMessage"]</div>
}
```

`TempData` is read-once by default—after you read it, it's marked for deletion. You can also use `TempData.Peek()` to read without marking, and `TempData.Keep()` to retain it for another request.

**Comparison Table:**

| Method         | Scope            | Data Type | Compile-time Check | Use Case                                  |
|----------------|------------------|-----------|---------------------|-------------------------------------------|
| Strong Model   | Single request   | Any       | Yes                 | Main data for view                        |
| ViewData       | Single request   | Object    | No                  | Auxiliary data, legacy compatibility      |
| ViewBag        | Single request   | dynamic   | No                  | Simple, quick data                        |
| TempData       | Two requests     | Object    | No                  | Messages after redirect, wizard steps     |

**Best Practice:** Prefer strongly-typed models for the primary data. Use `ViewData`/`ViewBag` sparingly for page-specific trinkets. Use `TempData` for cross-request notifications.

---

### 4.5 Tag Helpers vs. HTML Helpers

Tag Helpers and HTML Helpers are two ways to generate HTML in Razor views. Tag Helpers are the modern, recommended approach.

#### HTML Helpers (Legacy, but still used)

HTML Helpers are methods that return HTML strings. They are called using Razor syntax with `@`:

```html
@Html.ActionLink("Home", "Index", "Home")
@Html.TextBoxFor(m => m.Name)
```

They produce HTML like `<a href="/Home/Index">Home</a>` and `<input id="Name" name="Name" type="text" value="" />`.

HTML Helpers were the primary way in older ASP.NET MVC, but they have drawbacks:
- Syntax is not HTML-like; you have to learn helper names.
- Difficult to add custom attributes (though you can pass `htmlAttributes`).
- Less IntelliSense support for HTML attributes.

#### Tag Helpers (Modern)

Tag Helpers look like standard HTML tags but are processed on the server to add or modify attributes. They are more intuitive for web designers and developers familiar with HTML.

**Example:**
```html
<a asp-controller="Home" asp-action="Index">Home</a>
<label asp-for="Name"></label>
<input asp-for="Name" class="form-control" />
```

These Tag Helpers generate the same HTML as the HTML Helpers above, but the syntax is cleaner.

#### Built-in Tag Helpers

ASP.NET Core includes many built-in Tag Helpers:

- **Anchor Tag Helper** (`<a asp-controller="..." asp-action="...">`) – generates correct URLs.
- **Form Tag Helper** (`<form asp-action="..." asp-controller="...">`) – adds anti-forgery token automatically.
- **Input Tag Helper** (`<input asp-for="...">`) – binds to model properties and adds validation attributes.
- **Label Tag Helper** (`<label asp-for="...">`) – displays property display name.
- **Validation Message Tag Helper** (`<span asp-validation-for="...">`) – displays validation errors.
- **Select Tag Helper** (`<select asp-for="..." asp-items="...">`) – populates dropdowns.
- **Environment Tag Helper** (`<environment include="Development">`) – conditionally renders content based on environment.
- **Cache Tag Helper** (`<cache>` – caches content.

#### Enabling Tag Helpers

Tag Helpers are enabled via `_ViewImports.cshtml`. The default template includes:

```csharp
@addTagHelper *, Microsoft.AspNetCore.Mvc.TagHelpers
```

This imports all built-in Tag Helpers.

#### Creating a Custom Tag Helper

You can create your own Tag Helpers for reusable HTML generation. For example, a Tag Helper that renders a "Back" button:

```csharp
using Microsoft.AspNetCore.Razor.TagHelpers;

namespace MyFirstMvcApp.TagHelpers
{
    public class BackButtonTagHelper : TagHelper
    {
        public string? Text { get; set; } = "Back";
        public string? Controller { get; set; }
        public string? Action { get; set; }

        public override void Process(TagHelperContext context, TagHelperOutput output)
        {
            output.TagName = "a";
            output.Attributes.SetAttribute("href", $"/{Controller ?? "Home"}/{Action ?? "Index"}");
            output.Attributes.SetAttribute("class", "btn btn-secondary");
            output.Content.SetContent(Text);
        }
    }
}
```

Usage in view:
```html
<back-button controller="Products" action="Index" text="Go to Products" />
```

You must register the Tag Helper in `_ViewImports.cshtml`:
```csharp
@addTagHelper *, MyFirstMvcApp
```

**Comparison: Which to use?**

- **Tag Helpers** are preferred for all new development. They are more HTML-like, easier to read, and fully integrated with IntelliSense.
- **HTML Helpers** are still supported and may appear in legacy code. You can mix them, but strive to use Tag Helpers for new work.

---

### 4.6 View Imports (`_ViewImports.cshtml`)

The `_ViewImports.cshtml` file is another special Razor file. It's used to bring common directives into all views in a folder and its subfolders. Typically placed in the `Views` folder, it can contain:

- `@using` statements (to import namespaces).
- `@inject` statements (to inject services directly into views).
- `@addTagHelper` (to enable Tag Helpers).
- `@model` (to set a default model type—rarely used here).
- `@namespace` (to set the namespace for generated views).

**Example `_ViewImports.cshtml`:**
```csharp
@using MyFirstMvcApp
@using MyFirstMvcApp.Models
@addTagHelper *, Microsoft.AspNetCore.Mvc.TagHelpers
```

This ensures that every view can use Tag Helpers and refer to model types without fully qualifying namespaces.

---

### 4.7 Putting It All Together: Enhancing the Product Application

Let's extend our product example to use the features we've learned.

#### Add a Partial View for Product Card

We already created `_ProductCard.cshtml`. Now modify `Products/Index.cshtml` to use it:

```html
@model IEnumerable<Product>

<h1>Product List</h1>

<div class="row">
    @foreach (var product in Model)
    {
        <div class="col-md-4">
            <partial name="_ProductCard" model="product" />
        </div>
    }
</div>
```

#### Add a View Component for Cart Summary

Create the `CartSummaryViewComponent` as shown earlier. Then modify the layout (`_Layout.cshtml`) to include it in the navigation:

```html
<ul class="navbar-nav flex-grow-1">
    <li class="nav-item">
        <a class="nav-link text-dark" asp-controller="Home" asp-action="Index">Home</a>
    </li>
    <li class="nav-item">
        <a class="nav-link text-dark" asp-controller="Products" asp-action="Index">Products</a>
    </li>
</ul>
<!-- Add the cart summary here -->
@await Component.InvokeAsync("CartSummary")
```

Now every page will display the cart summary.

#### Use TempData for Success Message

In the `ProductsController`, add a `Create` action (POST) and use TempData:

```csharp
[HttpPost]
public IActionResult Create(Product product)
{
    if (ModelState.IsValid)
    {
        product.Id = _products.Max(p => p.Id) + 1;
        _products.Add(product);
        TempData["SuccessMessage"] = "Product added successfully!";
        return RedirectToAction(nameof(Index));
    }
    return View(product);
}
```

In `Index.cshtml`, display the message:

```html
@if (TempData["SuccessMessage"] != null)
{
    <div class="alert alert-success">@TempData["SuccessMessage"]</div>
}
```

#### Use Tag Helpers for Forms

We'll cover forms in depth in the next chapter, but here's a preview of a `Create.cshtml` view using Tag Helpers:

```html
@model Product

<h1>Create Product</h1>

<form asp-action="Create" method="post">
    <div class="form-group">
        <label asp-for="Name"></label>
        <input asp-for="Name" class="form-control" />
        <span asp-validation-for="Name" class="text-danger"></span>
    </div>
    <div class="form-group">
        <label asp-for="Price"></label>
        <input asp-for="Price" class="form-control" />
        <span asp-validation-for="Price" class="text-danger"></span>
    </div>
    <div class="form-group">
        <label asp-for="Description"></label>
        <textarea asp-for="Description" class="form-control"></textarea>
    </div>
    <button type="submit" class="btn btn-primary">Create</button>
    <a asp-action="Index" class="btn btn-secondary">Cancel</a>
</form>
```

---

### Summary

You've now mastered the tools to build rich, interactive, and maintainable Razor views:

- **Layouts** provide consistent page structure.
- **Partial views** allow reuse of UI snippets.
- **View components** encapsulate logic and UI together.
- **Data passing techniques** give you flexibility in moving data from controller to view.
- **Tag Helpers** modernize HTML generation with an intuitive, HTML-like syntax.

With these building blocks, you can create complex web pages that are easy to manage and extend.

**Exercise:**
1. Add a new `CategoriesController` and corresponding views.
2. Create a partial view `_CategoryCard.cshtml` that displays category name and description.
3. Create a view component `CategoryMenuViewComponent` that lists all categories (simulate data) and renders a dropdown or sidebar.
4. Display the category menu in the layout (e.g., in the header or sidebar).
5. Use `ViewBag` to pass a page title from controller to view, and `TempData` to show a message after adding a category.

In the next chapter, **"Working with Forms and User Input,"** you'll learn how to handle user input through forms, model binding, and validation—essential skills for any web application that accepts data from users.